In [1]:
import os
os.chdir('../')

In [18]:
from Emotion_Detector.utils import *
from Emotion_Detector.utils.model_store import LenetModel
from Emotion_Detector.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [19]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    callback_dir : Path
    tensorboard_log_dir : Path
    model_history_dir : Path
    model_dir : Path
    model_file_path : str
    callback_file_path : str
    model_history_file_path : Path
    train_tf_records_file_path : str
    test_tf_records_file_path : str
    param_image_size : int
    param_learning_rate : float
    param_epochs : int
    param_train_num_shards : int
    param_test_num_shards : int
    param_batch_size : int
    all_params : dict

In [20]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training


        create_directories([config.root_dir])
        create_directories([config.callback_dir])
        create_directories([config.tensorboard_log_dir])
        create_directories([config.model_history_dir])
        create_directories([config.model_dir])

        model_training_config = ModelTrainingConfig(
            root_dir = Path(config.root_dir),
            callback_dir = Path(config.callback_dir),
            tensorboard_log_dir = Path(config.tensorboard_log_dir),
            model_history_dir = Path(config.model_history_dir),
            model_dir = Path(config.model_dir),
            model_file_path =  config.model_file_path,
            callback_file_path = config.callback_file_path,
            model_history_file_path = Path(config.model_history_file_path),
            train_tf_records_file_path = config.train_tf_records_file_path,
            test_tf_records_file_path = config.test_tf_records_file_path,
            param_image_size = self.params.IMAGE_SIZE,
            param_learning_rate =  self.params.LEARNING_RATE,
            param_epochs = self.params.EPOCHS,
            param_train_num_shards = self.params.TRAIN_NUM_SHARDS,
            param_test_num_shards = self.params.TEST_NUM_SHARDS,
            param_batch_size = self.params.BATCH_SIZE,
            all_params = self.params
        )
        return model_training_config

In [21]:
class Model_Training:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def _get_data(self):
        train_data = reconstruct_data_from_tfrecords(path = self.config.train_tf_records_file_path,
                                                    num_shards= self.config.param_train_num_shards, 
                                                    batch_size = self.config.param_batch_size )
        val_data = reconstruct_data_from_tfrecords(path = self.config.test_tf_records_file_path,
                                                    num_shards= self.config.param_test_num_shards, 
                                                    batch_size = self.config.param_batch_size )
        return train_data, val_data
        
    def _get_model(self):
        input_shape = (1, self.config.param_image_size, self.config.param_image_size, 3)
        print(self.config.all_params)
        lenet_model = LenetModel(configurations = self.config.all_params)
        lenet_model.build(input_shape = input_shape)
        return lenet_model
    
    def train_model(self):

        train_data, val_data = self._get_data()
        model = self._get_model()

        checkpoint_callback = ModelCheckpoint(
            filepath= self.config.callback_file_path,
            save_weights_only=True,
            save_best_only=True,
            monitor='val_loss',
            verbose=1
        )

        tensorboard_callback = TensorBoard(log_dir=self.config.tensorboard_log_dir, histogram_freq=1)


        loss_function = SparseCategoricalCrossentropy()
        metrics = [SparseCategoricalAccuracy(name = "accuracy")]

        model.compile(
                optimizer = Adam(learning_rate= self.config.param_learning_rate),
                loss = loss_function,
                metrics = metrics
            )
        
        history = model.fit(train_data, epochs = self.config.param_epochs,validation_data = val_data, callbacks = [checkpoint_callback, tensorboard_callback], verbose = 1)

        save_json(path = self.config.model_history_file_path, data = history.history)

        model.save(self.config.model_dir, save_format = 'tf')

        print('Model Training Complete')

In [22]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_trainer = Model_Training(config = model_training_config)
    model_trainer.train_model()
except Exception as e:
    raise e

{'CLASS_NAMES': ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'], 'RANDOM_SEED': 22, 'BATCH_SIZE': 32, 'IMAGE_SIZE': 256, 'LEARNING_RATE': 0.001, 'EPOCHS': 3, 'DROPOUT_RATE': 0.0, 'REGULARIZATION_RATE': 0.0, 'N_FILTERS': 6, 'KERNAL_SIZE': 3, 'N_STRIDES': 1, 'POOL_SIZE': 2, 'N_DENSE_1': 100, 'N_DENSE_2': 10, 'NUM_CLASSES': 7, 'SHUFFLE': True, 'TRAIN_NUM_SHARDS': 10, 'TEST_NUM_SHARDS': 5, 'RANDOM_ROTATION_LEFT_FACTOR': -0.025, 'RANDOM_ROTATION_RIGHT_FACTOR': 0.025, 'RANDOM_FLIP_MODE': 'horizontal', 'RANDOM_CONTRAST_FACTOR': 0.1}
Epoch 1/3
    897/Unknown - 74s 81ms/step - loss: 1.8849 - accuracy: 0.2381
Epoch 1: val_loss improved from inf to 1.92501, saving model to artifacts/model_training/model_callbacks\model_01.ckpt
898/898 [==============================] - 84s 92ms/step - loss: 1.8848 - accuracy: 0.2381 - val_loss: 1.9250 - val_accuracy: 0.1686
Epoch 2/3
897/898 [============================>.] - ETA: 0s - loss: 1.5029 - accuracy: 0.4230
Epoch 2: val_loss did not

In [ ]:
from Emotion_Detector.utils import reconstruct_data_from_tfrecords

In [ ]:
import os
os.chdir('../')

In [ ]:
path = 'artifacts/data_preprocessing/train_tfrecords/shard_{:02d}.tfrecord'
num_shards = 10
batch_size = 32

In [ ]:
parsed_data = reconstruct_data_from_tfrecords(path, num_shards, batch_size )